# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


import sys
try:
    sys.path.insert(1,'/Users/Trav/Desktop/Coding/GitHub_Repositories/API_PACKAGE')
    from api_keys import g_key
except:
    from api_keys import g_key





### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
## import csv to dataframe, add try block to deal with directory issues
# csv_import_path = os.path.join('Weather_API','Weather','Export Folder','city_data.csv')
csv_import_path = os.path.join('..','Weather','Export Folder','city_data.csv')
city_weather=pd.read_csv(csv_import_path)

city_weather.head()


..\Weather\Export Folder\city_data.csv


,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,54.98,-117.91,valleyview,ca,"(55.066667, -117.283333)",55.066667,-117.283333,30.88,95,99,7.36
1,53.60,-13.37,dingle,ie,"(52.1408333, -10.2688889)",52.140833,-10.268889,52.36,83,100,10.71
2,-32.08,-16.69,jamestown,sh,"(-15.9333333, -5.7166667)",-15.933333,-5.716667,62.64,89,100,23.49
3,37.95,99.76,zhangye,cn,"(38.934167, 100.451667)",38.934167,100.451667,46.98,48,4,16.04
4,49.51,140.54,vanino,ru,"(49.086856, 140.254337)",49.086856,140.254337,44.83,64,24,6.89


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
